In [ ]:
import requests
import json
import os
from dotenv import load_dotenv

# Carica le variabili d'ambiente dal file .env
load_dotenv()

# Configurazione
BASE_URL = os.getenv("BASE_URL")
API_KEY = os.getenv("API_KEY")
SAVE_DIR = os.getenv("SAVE_DIR")
os.makedirs(SAVE_DIR, exist_ok=True)

def fetch_data(endpoint, filename, adjust_prices=False):
    """
    Scarica i dati dall'endpoint e salva in un file JSON.
    Se adjust_prices=True, divide i prezzi per 100.
    """
    url = f"{BASE_URL}/{endpoint}?output_format=JSON&display=full"
    response = requests.get(url, auth=(API_KEY, ''))
    
    if response.status_code == 200:
        data = response.json()
        
        # Se serve, corregge i prezzi
        if adjust_prices and endpoint == "order_details":
            for item in data:
                if "product_price" in item:
                    item["product_price"] = float(item["product_price"]) / 100
                if "unit_price_tax_incl" in item:
                    item["unit_price_tax_incl"] = float(item["unit_price_tax_incl"]) / 100
                if "total_price_tax_incl" in item:
                    item["total_price_tax_incl"] = float(item["total_price_tax_incl"]) / 100

        # Percorso di salvataggio
        file_path = os.path.join(SAVE_DIR, filename)
        
        # Salva il file
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        
        print(f"File salvato: {file_path}")
        return data
    else:
        print(f"Errore nel recupero dei dati da {endpoint}: {response.status_code}")
        return None

# Scarica gli ordini senza modifiche
print("Scarico ordini...")
fetch_data("orders", "orders.json")

# Scarica i dettagli degli ordini con prezzi corretti
print("Scarico dettagli ordini...")
fetch_data("order_details", "order_details.json", adjust_prices=True)

print("Script completato")